In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv('../input/dog-breed-identification/labels.csv')

In [ ]:
df1.shape

In [ ]:
df1.head()

In [ ]:
img_file='/kaggle/input/dog-breed-identification/train/'

df=df1.assign(img_path=lambda x: img_file + x['id'] +'.jpg')
print(df.shape)

In [ ]:
df.head()

In [ ]:
df['img_path'][0]

In [ ]:
from keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
X=np.array([img_to_array(load_img(img, target_size=(96, 96))) for img in df['img_path'].values.tolist()])
X.shape

In [ ]:
Y=pd.get_dummies(df['breed'])
Y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,Y_train,Y_val = train_test_split(X,Y,test_size=0.3)

print(X_train.shape,Y_train.shape)
print(X_val.shape,Y_val.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Conv2D,Flatten,MaxPool2D,Dropout

In [ ]:
model=Sequential()

model.add(Conv2D(64,(3,3),input_shape=(96,96,3)))
model.add(Activation('relu'))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(16,(3,3)))
model.add(Activation('relu'))

# model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(8,(3,3)))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dropout(0.25))

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dropout(0.25))

model.add(Dense(Y.shape[1]))
model.add(Activation('softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(X_train,Y_train,
         validation_data=(X_val,Y_val),
         batch_size=32,
         epochs=75,
         verbose=2)

In [ ]:
test_datagen = ImageDataGenerator ( 
    rescale=1/255.
)

test_set = test_datagen.flow_from_directory (
    '/kaggle/input/dog-breed-identification',
    target_size = (96,96),
    batch_size = 128,
    classes=['test']
)

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred.shape

In [ ]:
df_submission = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')
df_submission

In [ ]:
import re
file_list = test_set.filenames
id_list = []
for name in file_list:
    m = re.sub('test/', '', name)
    m = re.sub('.jpg', '', m)
    id_list.append(m)
# print(id_list)

In [ ]:
df_submission['id'] = id_list
df_submission.iloc[:,1:] =y_pred
df_submission.head()

In [ ]:
final_df = df_submission.set_index('id')
final_df.to_csv('Submission.csv')